<span style="font-size:110%; text-align:left;"><b><a href="../index.html">5. Tips and Examples</a> > <a href="./index.html">2. General modelling</a></b></span>
<br/><br/>
<span style="font-size:200%; text-align:left;"><b>Comparing error signals and transfer functions</b></span>
<br/><br/>
<span style="font-size:110%; text-align:left;">Author: Daniel Brown </span>

This is a short note on comparing the DC limit of a transfer functions and an error signal. At the DC limit these values should be the same, essentially how much power change do I see on my photodiode when I change some parameter in the interferometer.

It is good to check that these match at times to make sure you're doing the correct model. This can be done easily enough with Pykat, the main issue is using the right commands and making sure you know what the units are.

Below we take an example (That Hang-yu provided regarding the same question) that is looking at a PDH error signal for the end mirror motion. 

In [1]:
import pykat
import numpy as np

kat=pykat.finesse.kat()

cmd="""
l Laser 1 0 0 n_l
s s1 0 n_l n_eom1
mod EOM 5M 0.3 1 pm 0 n_eom1 n_eom2
s s2 0 n_eom2 n_m11
m1 M1 400u 0 0 n_m11 n_m12
s s_cav 100 n_m12 n_m21
m1 M2 0 0 0 n_m21 n_m22

pd1 PD 5M 0 n_m11
pd2 PD_ac 5M 0 $fs n_m11
yaxis re:im
noxaxis
"""

                                              ..-
    PyKat 1.1.349         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



Above is the basic model. Next we want to run the model twice, once for the DC error signal slope, then again with the transfer function. We compute the error signal slope using the `diff` command which automatically differentiates all the outputs in the simulation with respect to `M2` tuning `phi`. This slope has units of Watts-per-degree, as the `phi` tuning parameter is in degrees.

The transfer function is done using the `fsig` command, targetting `M2` phase, which is in units of radians. We do this for a suitable low frequency of 1µHz. In practice you should plot the full tranfer function over frequency to make sure you are at a low enough value.

In [2]:
kat.parse(cmd)
kat.verbose=False

cmd_list=[
    """
    diff M2 phi
    deriv_h 1e-10
    """, 
    """
    fsig sig1 M2 phase 1e-6 0 1
    """]

We now run the model and output each result:

In [3]:
for i in range(2):
    kat_loop=kat.deepcopy()
    kat_loop.parse(cmd_list[i])
    out=kat_loop.run()
    
    if i==0:
        print('DC resp')
        print(out['PD'], '[W/deg]')
    else:
        print('TF at DC')
        print(out['PD_ac'], '[W/rad]')
        print(out['PD_ac'] * np.pi / 180, '[W/deg]')

DC resp
(-101.20910440502+0j) [W/deg]
TF at DC
(-5798.85453070566+0.000121510697403016j) [W/rad]
(-101.2091044050044+2.1207617460771524e-06j) [W/deg]


As we can see, they match up well after correcting for unit conversions.